In [ ]:
from google import genai 
client = genai.Client()

payload= "give me the data quality steps that I need to have for this table -- `551face`.check_table definition CREATE TABLE `check_table` ( `contract_id` int DEFAULT NULL, `name` varchar(20) DEFAULT NULL, CONSTRAINT `ck_name` CHECK ((`contract_id` > 100))) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;"

In [ ]:
response = client.models.generate_content(model='gemini-2.0-flash', contents=payload);

In [13]:
context = {
    "table_name": "check_table",
    "database": "551face",
    "schema": [
        {"name": "contract_id", "type": "int", "nullable": True},
        {"name": "name", "type": "varchar(20)", "nullable": True},
    ],
    "constraints": [
        {
            "name": "ck_name",
            "type": "CHECK",
            "expression": "contract_id > 100"
        }
    ]
}


In [14]:
import json
from google import genai
client = genai.Client()

request_obj = {
    "instruction": "You are a senior data engineer. Given this table, propose data-quality tests, a data contract, and markdown docs.",
    "context": context,
    "output_format": {
        "Data Quality": "contract spec for the table without missing a single thing",
        "post load checks": "list of test definitions",
        "docs_markdown": "markdown description of the table very clearly"
    }
}

contents = f"""
You are a senior data engineer.

Here is a JSON with table context:

{json.dumps(request_obj, indent=2)}

Return a JSON object with the following top-level keys only:
- "tests": list of tests, each with name, description, and SQL or rule.
- "Data Quality": data contract definition (keys, nullability, rules, checks) include everything relavant to the given tables definations for each table respectively.
- "docs_markdown": markdown string describing the table.

Do NOT return explanations outside the JSON, only the JSON object.
"""

response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=contents,
)

print(response)


sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""```json
{
  "tests": [
    {
      "name": "contract_id_is_not_null",
      "description": "The primary identifier for a contract should always be present.",
      "sql": "SELECT COUNT(*) FROM 551face.check_table WHERE contract_id IS NULL"
    },
    {
      "name": "contract_id_is_unique",
      "description": "Each contract must have a unique identifier.",
      "sql": "SELECT contract_id FROM 551face.check_table GROUP BY contract_id HAVING COUNT(*) > 1"
    },
    {
      "name": "contract_id_adheres_to_check_constraint",
      "description": "Ensures all contract_id values are greater than 100 as per the table constraint.",
      "sql": "SELECT COUNT(*) FROM 551face.check_table WHERE contract_id <= 100"
    },
    {
      "name": "name_is_not_null",
      "description": "The contract name should not be null or empty.",
      "sql": "SELECT COUN